<a href="https://colab.research.google.com/github/aszapla/Curso-DL/blob/master/3_1_Teoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 3.1: redes recurrentes avanzadas

Profesor: [Jorge Calvo Zaragoza](mailto:jcalvo@prhlt.upv.es)

## Resumen
En esta sesión:
  * Repasaremos arquitecturas neuronales avanzadas con redes recurrentes.
  * Veremos problemas complejos que utilizan estas estructuras. 
  * Programaremos paso a paso un traductor automático.

## Introducción

Dependiendo de la tipología de las entradas y las salidas, podemos formular diferentes escenarios de aprendizaje automático:

![Redes recurrentes](https://www.microsoft.com/en-us/cognitive-toolkit/wp-content/uploads/sites/3/2017/04/071316_1312_RecurrentNe3.jpg)

* **One to one**: clasificación convencional en el cual a una entrada le corresponde una salida.
* **One to many**: tareas en las cuáles se quiere obtener una secuencia a partir de una única entrada. Ejemplo: descripción automática de imagen.
* **Many to one**: clasificación de secuencias, en las cuales queremos asignar una categoria a una secuencia. Ejemplo: análisis de sentimiento.
* **Many to many**: tareas en las cuales tanto la entrada como la salida son secuencias. Se puede plantear de diferente forma dependiendo de si las secuencias son desacopladas (traducción automática) o si hay una fuerte relación entre cada elemento de la entrada y la salida (etiquetado gramatical).

Ya se ha visto en la anterior sesión que las redes recurrentes son adecuadas para tratar con problemas de naturaleza secuencial. Keras contiene la implementación tres tipos de redes recurrentes, añadiendo una interfaz para desarrollar redes recurrentes a medida:

* **[SimpleRNN](https://keras.io/layers/recurrent/#simplernn)**
* **[LSTM](https://keras.io/layers/recurrent/#lstm)**
* **[GRU](https://keras.io/layers/recurrent/#gru)**

![texto alternativo](http://sqlml.azurewebsites.net/wp-content/uploads/2017/11/null-9.png)

La flexibilidad para construir las distintas topologías mostradas en la figura anterior se proporcionan a través de siguientes parámetros:

* **return_sequences**: cuando está desactivado, sólo devuelve la salida del último instante de tiempo (many to one); cuando está activado, devuelve la salida de todos los instantes (many to many).
* **return_state**: indica si se accede a los estados recurrentes de las neuronas.



In [0]:
from keras.layers import Input
from keras.layers import SimpleRNN, LSTM, GRU

# La entrada a una red recurrente tiene dos dimensiones:
# - El número de instantes de tiempo (None para indicar que variable)
# - Número de características de entrada
T = 100
feature_dim = 2
input_layer = Input(shape=(T, feature_dim))
  
# LSTM
x0 = LSTM(64)(input_layer)
x1 = LSTM(64,return_state=True)(input_layer)
x2 = LSTM(64,return_sequences=True)(input_layer)
x3 = LSTM(64,return_sequences=True,return_state=True)(input_layer)
print('LSTM: ' + str(x0))
print('LSTM + return_state: ' + str(x1))
print('LSTM + return_sequences: ' + str(x2))
print('LSTM + return_sequences + return_state: ' + str(x3))
print()




Using TensorFlow backend.


LSTM: Tensor("lstm_1/TensorArrayReadV3:0", shape=(?, 64), dtype=float32)
LSTM + return_state: [<tf.Tensor 'lstm_2/TensorArrayReadV3:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_2/while/Exit_2:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_2/while/Exit_3:0' shape=(?, 64) dtype=float32>]
LSTM + return_sequences: Tensor("lstm_3/transpose_1:0", shape=(?, ?, 64), dtype=float32)
LSTM + return_sequences + return_state: [<tf.Tensor 'lstm_4/transpose_1:0' shape=(?, ?, 64) dtype=float32>, <tf.Tensor 'lstm_4/while/Exit_2:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_4/while/Exit_3:0' shape=(?, 64) dtype=float32>]



Como veremos a continuación, la posibilidad de obtener los estados internos de la red recurrente es lo que nos permite construir una arquitectura secuencia a secuencia desacoplada.

#### Recurrencia bidireccional

En la mayoría de tareas, es interesante tener no sólo el contexto anterior sino también el posterior a la hora de predecir un elemento en un instante dado. En tal caso, podemos establecer una **recurrencia bidireccional**, en las cuales una neurona tiene dos conexiones recurrentes que, **a la hora de desplegarse**, lo hacen en direcciones opuestas.

![texto alternativo](https://cdn-images-1.medium.com/max/764/1*6QnPUSv_t9BY9Fv8_aLb-Q.png)

Para implementar redes bidireccionales Keras aporta un *wrapper* que acepta como parámetro un objeto que implemente la super clase recurrente:

* [keras.layers.Bidirectional](https://keras.io/layers/wrappers/#bidirectional)

In [0]:
from keras.layers import Input
from keras.layers import LSTM, Bidirectional

T = None
feature_dim = 2

# BLSTM
input_layer = Input(shape=(T, feature_dim))
x0 = Bidirectional(LSTM(64))(input_layer)
x1 = Bidirectional(LSTM(64,return_state=True))(input_layer)
x2 = Bidirectional(LSTM(64,return_sequences=True))(input_layer)
x3 = Bidirectional(LSTM(64,return_sequences=True,return_state=True))(input_layer)

print('BLSTM: ' + str(x0))
print('BLSTM + return_state: ' + str(x1))
print('BLSTM + return_sequences: ' + str(x2))
print('BLSTM + return_sequences + return_state: ' + str(x3))


Using TensorFlow backend.


BLSTM: Tensor("bidirectional_1/concat:0", shape=(?, 128), dtype=float32)
BLSTM + return_state: [<tf.Tensor 'bidirectional_2/concat:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'bidirectional_2/while/Exit_2:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'bidirectional_2/while/Exit_3:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'bidirectional_2/while_1/Exit_2:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'bidirectional_2/while_1/Exit_3:0' shape=(?, 64) dtype=float32>]
BLSTM + return_sequences: Tensor("bidirectional_3/concat:0", shape=(?, ?, 128), dtype=float32)
BLSTM + return_sequences + return_state: [<tf.Tensor 'bidirectional_4/concat:0' shape=(?, ?, 128) dtype=float32>, <tf.Tensor 'bidirectional_4/while/Exit_2:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'bidirectional_4/while/Exit_3:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'bidirectional_4/while_1/Exit_2:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'bidirectional_4/while_1/Exit_3:0' shape=(?, 64) dtype=float32>]



## Arquitectura secuencia a secuencia

De entre todos los enfoques mostrados en el bloque anterior, el más flexible es el de secuencia a secuencia desacoplado (*sequence to sequence*, o *seq2seq*). Típicamente, se implementa con una arquitectura encoder-decoder:

![texto alternativo](https://docs.google.com/uc?id=1poBXbaLFiEN0IPZtR0IIbySEnSLGRvih)

Esta arquitectura cuenta con dos redes neuronales independientes:
* El **encoder** es una red recurrente que se encarga de procesar la entrada elemento a elemento, almacenando en su estado interno una codificación compacta y representativa de la información procesada hasta el momento. Al estado interno de las neuronas del encoder se le llama *context vector* o *thought vector*.
* El **decoder** es otra red recurrente que recibe el *context vector* de la última etapa del encoder. En cada paso, predice un elemento del dominio de salida, utilizando el estado interno de la red recurrente y el último elemento predicho. El proceso acaba cuando se emite el elemento especial *EOS* (end of sentence).

![texto alternativo](https://devblogs.nvidia.com/wp-content/uploads/2015/06/Figure6_summary_vector_space.png)

Esta arquitectura se entrena de manera conjunta, para lo cual tan sólo hacen falta pares de secuencias de entrada y salida, sin necesidad de proporcionar ningun tipo de información acerca de la relación entre los elementos de las mismas.

## Ejemplos de tareas

Una ventaja de esta arquitectura es que sirve para multitud de tareas que en el pasado se desarrollaban de forma independiente. Aunque cada tarea tiene sus particularidades, esta formulación similar permite que avances significativos en un campo sean generalizables a otro (como veremos más adelante con los modelos de atención).

La única restricción para llevar a cabo la mayoría de estas tareas son de índole logístico: ¿tenemos suficientes datos para aprender dicha tarea? ¿tenemos suficiente capacidad computacional para llevar a cabo el entrenamiento? 


### Modelos de conversación

![texto alternativo](https://cdn-images-1.medium.com/max/1585/1*sO-SP58T4brE9EHazHSeGA.png)


### Traducción automática

![texto alternativo](https://www.wncc-iitb.org/images/semantics.png)


### Resumen de texto

![texto alternativo](https://cdn-images-1.medium.com/max/1600/1*Cu49wPEpWJPoI0a5AV9Q1Q.png)




## Codificación de elementos no numéricos

Hemos visto que en los problemas anteriores, los elementos de entrada no son características numéricas (como pasa en el aprendizaje automático clásico o con las redes convolucionales) sino elementos discretos como caracteres o palabras. Si una red neuronal sólo *entiende* de números, ¿cómo podemos indicarle este tipo de características?







### One-hot encoding

El *one-hot encoding* es una representación numérica de variables discretas. Consiste en asumir vectores de características de dimension igual al número de diferentes valores del conjunto discreto de la variable a codificar. Cada dimension corresponde a un único elemento de dicho conjunto.

In [0]:
import numpy as np

# Imaginemos un vocabulario de colores
vocabulary = {'rojo', 'amarillo', 'azul', 'verde', 'lila' ,'naranja'}

# Asignamos a cada caracter un índice numérico
word2int = dict([(char, i) for i, char in enumerate(vocabulary)])
print('Asignacion de indice a palabras')
print(word2int)

# De esta forma las frases se componen de secuencias de vectores de 6 elementos
sentence = 'rojo amarillo naranja'
tokenized_sentence = sentence.split()
encoded_sentence = np.zeros([len(tokenized_sentence),len(vocabulary)])

# Para cada palabra, activamos la posición que corresponde a su identificador 
for i,c in enumerate(sentence.split()):
  encoded_sentence[i][ word2int[c] ] = 1

# Comprobación
print('')
print('Frase original: ' + sentence)

print('Frase secuencial:')
print(str(tokenized_sentence))

print('Frase codificada:')
print(str(encoded_sentence))

Asignacion de indice a palabras
{'azul': 0, 'amarillo': 1, 'rojo': 2, 'naranja': 3, 'verde': 4, 'lila': 5}

Frase original: rojo amarillo naranja
Frase secuencial:
['rojo', 'amarillo', 'naranja']
Frase codificada:
[[0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


El one-hot encoding tiene una ventaja importante: no asume ninguna relación entre los elementos del vocabulario. Dicho de otro modo, todos los elementos del conjunto son equidistantes entre sí en la codificación one-hot (distancia hamming 2). Por otra parte, el one-hot encoding tiene la máxima redundancia; en realidad, para codificar N elementos, tan sólo harían falta vectores de log(N) bits.

El problema es que si ordenamos cada elemento del vocabulario (por ejemplo, por orden alfabético) y le asignamos su posición codificándola en binario, le estaríamos indicando de alguna forma a la red neuronal que los elementos que están cerca en dicho orden es porque tienen un rol más similar (lo cual no es cierto en la mayoría de las ocasiones).





#### Estado actual

A menudo encontramos tareas para las cuales el vocabulario a tener en cuenta es inmenso. Por ejemplo, el idioma español tiene alrededor de 100.000 vocablos distintos. Sin embargo, ya hemos comentado que utilizar un enfoque más compacto puede ser perjudicial para el proceso de aprendizaje. Además, ¿qué ocurre con las palabras muy poco frecuentes o que incluso están fuera del vocabulario que habíamos planeado inicialmente? 

Una posibilidad para lidiar con este escenario es bajar a nivel de caracteres, cuyo vocabulario suele ser mucho más limitado. El problema es que la red neuronal tiene que hacer un esfuerzo mayor en inferir las relaciones entre los distintos elementos de la entrada. En la actualidad, es común utilizar un enfoque intermedio basado en sub-palabras.

#### Keras

Keras cuenta con utilidades específicas para tratar con texto y codificaciones one-hot:


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence

# Lista de frases
corpus = ["El rojo y el amarillo hacen naranja.",
     "El lila viene del azul y del rojo.",
     "El azul es primario",
     "El naranja es secundario"]

# Objeto tokenizer
tokenizer = Tokenizer()

# Ajustamos el objeto al corpus
tokenizer.fit_on_texts(corpus)

# Comprobación de la asignación de un índice a cada palabra
print('Asignacion de indice a palabras')
print(tokenizer.word_index)

# Comprobamos el número de palabras
num_words = len(tokenizer.word_index)
print('')
print('Número de palabras: ' + str(num_words))

# Codificamos una frase
sentence = 'rojo amarillo naranja'

# La separamos en palabras
tokenized_sentence = text_to_word_sequence(sentence)

# La codificamos siguiendo one-hot
encoded_sentence = tokenizer.texts_to_matrix(tokenized_sentence)

# Comprobación
print('')
print('Frase original: ' + sentence)
print('')
print('Frase secuencial:')
print(str(tokenized_sentence))
print('')
print('Frase secuencial:')
print(str(encoded_sentence))




Asignacion de indice a palabras
{'el': 1, 'rojo': 2, 'y': 3, 'naranja': 4, 'del': 5, 'azul': 6, 'es': 7, 'amarillo': 8, 'hacen': 9, 'lila': 10, 'viene': 11, 'primario': 12, 'secundario': 13}

Número de palabras: 13

Frase original: rojo amarillo naranja

Frase secuencial:
['rojo', 'amarillo', 'naranja']

Frase secuencial:
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


## Embedding

Como acabamos de comentar, el one-hot encoding permite al usuario proporcionar a la red una entrada agnóstica, en la cual cada elemento tiene una representación igualmente independiente. Obviamente, para una tarea específica, esta asunción no es valida (ni útil): hay palabras que tienen un rol o un significado similar y por tanto, sería adecuado que tuvieran una representación interna similar. 

Idealmente, queremos representar cada palabra mediante un vector numérico en el cual las palabras similares (en rol o en significado) esten cerca en ese espacio. Esto se conoce como *embedding*.  Siguiendo los principios del deep learning, queremos que la red aprenda estas relaciones por sí sola en lugar de establecer el embedding siguiendo reglas heurísticas o basadas en nuestra propia intuición. 




### word2vec

La idea de realizar realizar embeddings es muy antigua. Sin embargo, su popularidad se incrementó considerablemente a partir del surgimiento de los modelos *word2vec*.


#### Idea 

![texto alternativo](https://deeplearning4j.org/img/word2vec_diagrams.png)

![texto alternativo](http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/img/Bottou-WordSetup.png)

#### Análisis

![texto alternativo](http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/img/Colbert-WordTable2.png)

#### Representación semántica

[Visualizacion](http://metaoptimize.s3.amazonaws.com/cw-embeddings-ACL2010/embeddings-mostcommon.EMBEDDING_SIZE=50.png)

![texto alternativo](http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/img/Mikolov-GenderVecs.png)

### Capas de embedding en Keras

En Keras, la capa de embedding no se hace a través de operaciones matemáticas. En su lugar, lo que se hace es acceder a una tabla look-up que asocia cada entero con una posición. La capa Embedding de Keras recibe directamente el entero que identifica un elemento del vocabulario (sin necesidad de hacer un one-hot encoding) y devuelve su representación densa. 

In [0]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Embedding

# Definimos el tamaño del vocabulario y las dimensiones del espacio latente
vocabulary_size = 5
embedding_space = 3

# Definimos una entrada de longitud variable
input_layer = Input(shape=(None,), dtype='int32', name='main_input')

# Añadimos la capa de embedding
embedded = Embedding(vocabulary_size, embedding_space)(input_layer)

# Creamos un modelo a partir de estas dos capas
model = Model(input_layer,embedded)

# Probamos a 'predecir' a través de esta red 
integer_encoding = [4,1,3,3,3]
output_array = model.predict(np.asarray([integer_encoding]))

print('Representación de: ' + str(integer_encoding))
print(output_array)

Representación de: [4, 1, 3, 3, 3]
[[[-0.01578005  0.03436567 -0.00560244]
  [-0.04476501  0.00464042 -0.04337955]
  [ 0.02889243 -0.01887957  0.02207902]
  [ 0.02889243 -0.01887957  0.02207902]
  [ 0.02889243 -0.01887957  0.02207902]]]


A pesar de esta implementación, los valores de esta tabla se aprenden de manera específica para la tarea en cuestión con los medios convencionales de entrenamiento de redes neuronales.

A menudo es interesante importar los embeddings obtenidos mediante un word2vec de un dominio similar para utilizarlo como punto de partida en nuestra tarea. Keras facilita este escenario permitiendo establecer los pesos iniciales de la capa mediante el parámetro *weights*. Además, nos permite especificar si queremos modificar los pesos en nuestro propio entrenamiento o no.



In [0]:
from keras.models import Model
from keras.layers import Input, Embedding
import numpy as np

# Definimos el tamaño del vocabulario y las dimensiones del espacio latente
vocabulary_size = 5
embedding_space = 3

# Creamos una matriz de embedding aleatoria
embedding_matrix = np.random.rand(vocabulary_size,embedding_space)

print('Tabla de embedding inicial')
for idx, representation in enumerate(embedding_matrix):
  print(idx,representation)

# Definimos una entrada de longitud variable
input_layer = Input(shape=(None,), dtype='int32', name='main_input')

# Proporcionamos la matriz anterior como parámetro
embedding = Embedding(vocabulary_size, embedding_space,
                            weights=[embedding_matrix],
                            trainable=True)(input_layer)

# Creamos el modelo de embedding
model = Model(input_layer,embedding)

# Comprobamos el embedding sobre este modelo 
integer_encoding = [4,1,3,3,3]
output_array = model.predict(np.asarray([integer_encoding]))

print('')
print('Representación de: ' + str(integer_encoding))
print(output_array)

Tabla de embedding inicial
0 [0.78987165 0.02071891 0.44158715]
1 [0.51946222 0.92426688 0.92094709]
2 [0.57110801 0.49882486 0.68064374]
3 [0.22824449 0.33314266 0.09556879]
4 [0.36113611 0.48007591 0.0550137 ]

Representación de: [4, 1, 3, 3, 3]
[[[0.3611361  0.4800759  0.0550137 ]
  [0.5194622  0.9242669  0.9209471 ]
  [0.22824448 0.33314267 0.09556879]
  [0.22824448 0.33314267 0.09556879]
  [0.22824448 0.33314267 0.09556879]]]


Aunque la forma de importar la matriz de embedding no está oficialmente documentada en la página web de Keras, se puede acceder a un tutorial completo:

[Importando pesos pre-entrenados a una capa de Embedding](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html)

## Modelo de atención

A menudo, la etapa de codificación es muy compleja, especialmente cuando la secuencia de entrada es muy larga. Es por ello que el context vector es insuficiente para capturar la información necesaria para toda la etapa de decodificación. Para paliar este fenómeno, se utilizan los **modelos de atención**.

Un modelo de atención es una estructura neuronal que complementa el enfoque secuencia a secuencia. En cada paso de decodificación, el modelo de atención asigna un peso a cada uno de los elementos de la etapa de codificación; es decir, ayuda a saber en qué parte de la entrada debe el decodificador *poner su atención* en cada paso:

![texto alternativo](https://github.com/tensorflow/nmt/raw/master/nmt/g3doc/img/attention_mechanism.jpg)

Como subproducto de este proceso, el modelo aprende de manera suavizada tanto la tarea en sí como el alineamiento subyacente:

![texto alternativo](https://github.com/tensorflow/nmt/raw/master/nmt/g3doc/img/attention_vis.jpg)

Los modelos de atención representan el estado del arte en las tareas de secuencia a secuencia implementadas con un enfoque encoder-decoder, y son imprescindibles para construir exitosamente modelos neuronales para problemas complejos como la traducción automática.

### Modelo de atención en Keras

Actualmente, Keras no incorpora modelos de atención en sus implementaciones base. Para poder utilizar este tipo de esquemas, hay que hacerlo combinando los elementos que Keras sí incorpora.
* [Tutorial para incorporar atención en Keras](https://wanasit.github.io/attention-based-sequence-to-sequence-in-keras.html)